In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [ ]:
import numpy as np
import torch
print(f"NumPy: {np.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")


NumPy: 2.0.2
PyTorch: 2.6.0+cu124
CUDA Available: True


In [ ]:
# Step 2: Robust Data Loading
# ---------------------------
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
from io import StringIO
import requests


def load_ag_news():
    sources = [
        lambda: load_dataset("ag_news"),
        lambda: pd.read_csv("https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv", header=None, names=["label", "title", "text"]),
        lambda: pd.read_csv(StringIO(requests.get("https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv").text), header=None, names=["label", "title", "text"])
    ]
    for source in sources:
        try:
            data = source()
            if isinstance(data, pd.DataFrame):
                return {"train": Dataset.from_pandas(data)}
            return data
        except Exception:
            continue
    raise Exception("All data sources failed")


In [ ]:
# Load dataset
raw_dataset = load_ag_news()
if isinstance(raw_dataset, dict) and "train" in raw_dataset:
    raw_dataset["train"] = raw_dataset["train"].map(lambda x: {"label": x["label"] - 1})
    if "test" in raw_dataset:
        raw_dataset["test"] = raw_dataset["test"].map(lambda x: {"label": x["label"] - 1})
else:
    raw_dataset = raw_dataset.map(lambda x: {"label": x["label"] - 1})

# Reduce dataset size for faster training/debugging
MAX_SAMPLES = 20000
raw_dataset["train"] = raw_dataset["train"].shuffle(seed=42).select(range(MAX_SAMPLES))

# Ensure train/test split
split_dataset = raw_dataset["train"].train_test_split(test_size=0.1, seed=42)
dataset = DatasetDict({
    "train": split_dataset["train"],
    "test": split_dataset["test"]
})

print("\nData loaded successfully!")
print(f"Training samples: {len(dataset['train'])}")
print(f"Test samples: {len(dataset['test'])}")
print(f"First sample: {dataset['train'][0]}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]


Data loaded successfully!
Training samples: 18000
Test samples: 2000
First sample: {'label': 1, 'title': 'Arsenal matches record of 42 league games without a loss', 'text': 'Defending champion Arsenal matched a 25-year-old record of 42 league games without a loss in the top tier of English soccer by rallying past Middlesbrough 5-3 Sunday.'}


In [ ]:
# Step 3: Tokenization
# ---------------------------
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_datasets = {
    "train": dataset["train"].map(tokenize_function, batched=True),
    "test": dataset["test"].map(tokenize_function, batched=True)
}

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
# Step 4: Model Setup
# ---------------------------
from transformers import BertForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=4,
    id2label={0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}
).to(device)

print(f"\nModel loaded on {device}")


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Model loaded on cuda


In [ ]:
# Step 5: Training Setup
# ---------------------------
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
import transformers

print(f"Transformers version: {transformers.__version__}")

# Temporarily removing evaluation_strategy to diagnose the issue
training_args = TrainingArguments(
    output_dir="./results",
    # evaluation_strategy="epoch", # Removed for debugging
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="epoch",
    save_strategy="no",
    report_to="none",
    load_best_model_at_end=False,
    no_cuda=not torch.cuda.is_available()
)

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds)
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

Transformers version: 4.53.2


In [ ]:
# Step 6: Training & Evaluation
# ---------------------------
print("\nStarting training...")
try:
    trainer.train()
    results = trainer.evaluate()
    print(f"\nFinal Evaluation - Accuracy: {results['eval_accuracy']:.4f}, Loss: {results['eval_loss']:.4f}")
except Exception as e:
    print(f"\nTraining error: {e}")
    if "CUDA" in str(e):
        print("Trying CPU fallback...")
        device = torch.device("cpu")
        model = model.to(device)
        trainer = Trainer(
            model=model,
            args=TrainingArguments(
                output_dir="./results_cpu",
                evaluation_strategy="epoch",
                learning_rate=2e-5,
                per_device_train_batch_size=2,
                num_train_epochs=3,
                no_cuda=True
            ),
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets["test"],
            compute_metrics=compute_metrics,
        )
        trainer.train()
        results = trainer.evaluate()
        print(f"\nFinal Evaluation (CPU) - Accuracy: {results['eval_accuracy']:.4f}, Loss: {results['eval_loss']:.4f}")



Starting training...


Step,Training Loss
4500,0.425300
9000,0.246900
13500,0.133900



Final Evaluation - Accuracy: 0.9190, Loss: 0.4468


In [ ]:
# Step 7: Prediction
# ---------------------------
if 'trainer' in locals():
    def predict(text):
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
        return model.config.id2label[logits.argmax().item()]

    test_samples = [
        "Stock markets reached record highs today",
        "The national football team won the championship",
        "Scientists discovered a new species in the Amazon",
        "Stock markets reached record highs today",
    "The national football team won the championship",
    "Scientists discovered a new species in the Amazon",
    "Apple unveiled a new AI-powered iPhone at its annual event",
    "The prime minister met with world leaders to discuss climate change",
    "NASA successfully landed a rover on Mars",
    "The basketball finals drew record viewership numbers",
    "Banking sector shows strong growth despite economic concerns",
    "Researchers are testing a new vaccine for malaria",
    "Olympic games are set to begin in Paris next month"
    ]

    print("\nTest Predictions:")
    for text in test_samples:
        print(f"{text[:50]}... → {predict(text)}")

    model.save_pretrained("./ag_news_model")
    tokenizer.save_pretrained("./ag_news_model")
    print("\nModel saved successfully!")
else:
    print("Training failed - cannot evaluate")


Test Predictions:
Stock markets reached record highs today... → Business
The national football team won the championship... → World
Scientists discovered a new species in the Amazon... → Sci/Tech
Stock markets reached record highs today... → Business
The national football team won the championship... → World
Scientists discovered a new species in the Amazon... → Sci/Tech
Apple unveiled a new AI-powered iPhone at its annu... → Sci/Tech
The prime minister met with world leaders to discu... → World
NASA successfully landed a rover on Mars... → Sci/Tech
The basketball finals drew record viewership numbe... → Sports
Banking sector shows strong growth despite economi... → Business
Researchers are testing a new vaccine for malaria... → World
Olympic games are set to begin in Paris next month... → Sports

Model saved successfully!
